*Verified by Leila Belabbassi (to work with Pangeo), July 12, 2018*

#### Note: This note book is a work in progress - There is a problem with the OpenDAP server behavior where the data live. 

In [ ]:
import requests
import re
import os
import xarray as xr

In [ ]:
url = 'https://opendap.oceanobservatories.org/thredds/catalog/ooi/friedrich.knuth@rutgers.edu/20171207T161702-RS03CCAL-MJ03F-05-BOTPTA301-streamed-botpt_nano_sample/catalog.html'
tds_url = 'https://opendap.oceanobservatories.org/thredds/dodsC'
datasets = requests.get(url).text
urls = re.findall(r'href=[\'"]?([^\'" >]+)', datasets)
x = re.findall(r'(ooi/.*?.nc)', datasets)
for i in x:
    if i.endswith('.nc') == False:
        x.remove(i)
for i in x:
    try:
        float(i[-4])
    except:
        x.remove(i)
datasets = [os.path.join(tds_url, i) for i in x]

In [ ]:
from dask_kubernetes import KubeCluster
cluster = KubeCluster(n_workers=20)
cluster

In [ ]:
from dask.distributed import Client
client = Client(cluster)
client

In [ ]:
%%time
ds = xr.open_mfdataset(datasets, decode_times=False, chunks={'obs': 10000}, parallel=True)

In [ ]:
ds = ds.swap_dims({'obs': 'time'})
ds